In [1]:
from torch import cuda, bfloat16
import transformers
import os
model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [4]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "How is the IP address configuration done for BGP peers, and why is it important?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How is the IP address configuration done for BGP peers, and why is it important? [/INST]  BGP (Border Gateway Protocol) is a routing protocol used to exchange routing information between autonomous systems (AS) on the internet. Here's how IP address configuration is done for BGP peers and why it's important:
1. IP Address Configuration:
To establish a BGP peering relationship, each peer must have a unique IP address that can be reached by the other peer. This is typically done by configuring the IP address of the BGP peer on the router that connects to the other peer. The IP address must be a valid IP address range for the autonomous system (AS) of the peer.
For example, if you want to establish a BGP peering relationship with a peer called "peer-as", you would configure the IP address of the BGP peer on your router as follows:
Router(config-router)# peering group 100
Router(config-router)# peer-as 100
In this example, the IP address of the BGP peer is 10.10.10.


In [5]:
import json
input_file_path = "/home/hb/LLM-research/evaluation/bgp_test_QA_val.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

Output: [INST]What is the purpose of BGP Update messages, and how do they carry route information?[/INST]  BGP (Border Gateway Protocol) Update messages are used to distribute routing information among autonomous systems (AS) on the internet.
The primary purpose of BGP Update messages is to inform other BGP peers about changes in the routing table of the originator AS. When a new route becomes available, or an existing route becomes invalid, the originator AS sends a BGP Update message to its peers to propagate the changes.
BGP Update messages carry route information in the following ways:
1. Route prefixes: BGP Update messages carry a list of route prefixes, along with their associated metrics (such as distance, AS path, and MED). Each route prefix is represented as a 4-byte IP address prefix followed by a 1-byte prefix length.
2. Route attributes: BGP Update messages can carry additional attributes about each route prefix, such as the originator AS, the type of route (e.g., internal,

/home/hb/myenv/lib/python3.8/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Output: [INST]What are BGP communities, and how can they be used for attribute manipulation in the BGP routing process?[/INST]  BGP (Border Gateway Protocol) communities are a mechanism used in the BGP routing process to filter and manipulate BGP updates based on specific attributes of the updates.
In BGP, each autonomous system (AS) has a unique AS number, and each BGP speaker (i.e., router) in that AS uses the AS number to identify and process BGP updates from other ASes. BGP communities are used to group BGP updates based on specific attributes, such as the originator of the update, the type of update, or the path that the update took to reach the current router.
BGP communities are specified using a set of tags or values that are associated with each BGP update. These tags or values are used to determine how the update should be processed and which paths the update should be advertised to.
There are several types of BGP communities that can be used for attribute manipulation in the